# API de Transcripción con Whisper en Google Colab

Este notebook crea una API pública que tu aplicación puede usar para transcribir audio.

## Instrucciones:

1. Ejecutar todas las celdas
2. Copiar la URL de ngrok que aparece
3. Actualizar la URL en tu aplicación de Netlify


In [ ]:
# Instalar dependencias
!pip install -q openai-whisper flask-ngrok flask flask-cors pyngrok


In [ ]:
# Importar librerías
import whisper
import os
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import tempfile

print('✅ Librerías importadas')


In [ ]:
# Cargar modelo Whisper
print('🔄 Cargando modelo Whisper small...')
model = whisper.load_model('small')
print('✅ Modelo Whisper cargado')


In [ ]:
# Crear API Flask
app = Flask(__name__)
CORS(app)  # Permitir CORS para todas las rutas

@app.route('/api/transcribe', methods=['POST', 'OPTIONS'])
def transcribe():
    # Manejar CORS preflight
    if request.method == 'OPTIONS':
        return '', 200
    
    try:
        print('🎵 Recibiendo archivo...')
        
        # Obtener archivo del request
        if 'audio' not in request.files:
            return jsonify({'success': False, 'error': 'No se recibió ningún archivo'}), 400
        
        audio_file = request.files['audio']
        print(f'📄 Archivo: {audio_file.filename}')
        
        # Guardar temporalmente
        with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as tmp_file:
            audio_file.save(tmp_file.name)
            temp_path = tmp_file.name
        
        print('🤖 Transcribiendo con Whisper...')
        
        # Transcribir
        result = model.transcribe(temp_path, language='es')
        
        # Limpiar archivo temporal
        os.unlink(temp_path)
        
        print('✅ Transcripción completada')
        
        return jsonify({
            'success': True,
            'text': result['text'],
            'duration': None,
            'size': None
        })
        
    except Exception as e:
        print(f'❌ Error: {str(e)}')
        return jsonify({'success': False, 'error': str(e)}), 500

@app.route('/health', methods=['GET'])
def health():
    return jsonify({'status': 'OK', 'message': 'API de transcripción funcionando'})

print('✅ API Flask creada')


In [ ]:
# Iniciar servidor con ngrok
print('🚀 Iniciando servidor Flask...')
print('📡 Creando túnel público con ngrok...')

# Crear túnel público
public_url = ngrok.connect(5000)
print(f'\n🔗 URL PÚBLICA: {public_url}')
print(f'📋 Copia esta URL y actualiza tu aplicación')
print(f'🎯 Endpoint completo: {public_url}/api/transcribe\n')

# Iniciar servidor
app.run(port=5000)
